In [ ]:
import pandas as pd
from difflib import SequenceMatcher

In [ ]:
list_keys = ['genres', 'production_countries', 'realisateur', 'casting', 'compagnies_production', 'langues']

def str_to_list(df, key):
    df_new = df

    for k in key: #on parcourt les clés à modifier spécifiées en argument de la fonction

        for ind in range(df_new.shape[0]): #on parcourt toutes les lignes du Dataframe

            string = df_new[k].iloc[ind] #on récupère la valeur à modifier pour chaque ligne

            if type(string) == str : # on vérifie si c'est bien une string
                
                #création de la nouvelle variable de type liste qui remplacera l'ancienne

                liste_finale = []

                #remplissage de la variable de type liste en faisant des opérations sur la string

                list_string = string[1:-1].split(',')
                for i,j in enumerate(list_string):
                    if i == 0:
                        mot = j[1:-1]
                        if mot not in liste_finale:
                            liste_finale.append(mot)
                    else:
                        mot = j[2:-1]
                        if mot not in liste_finale:
                            liste_finale.append(mot)
                
                #affectation de la nouvelle valeur de type liste à l'élément modifié du Dataframe

                df_new.at[ind,k] = liste_finale

    return df_new

In [ ]:
#Importation des datasets
df  = pd.read_csv("total.csv").drop(columns="Unnamed: 0") 
df = str_to_list(df, list_keys)
actor = pd.read_csv("data//actor_scrap.csv").drop(columns="Unnamed: 0")
director = pd.read_csv(r"data\director_scrap.csv").drop(columns="Unnamed: 0")

In [ ]:
#Contruction des notes des acteurs/réalisateur
#Acteur
actor["rating"] = 100*actor["Worldwide_box_office"]/actor['Worldwide_box_office'].max()
actor["rating"] = actor["rating"].astype(int)

#Réalisateur
director["rating"] = 100*director["Worldwide_box_office"]/director['Worldwide_box_office'].max()
director["rating"] = director["rating"].astype(int)


In [ ]:
#Création des dictionnaires
actor_dict = {}
for i, row in actor.iterrows():
    actor_dict[row.Actor_Name] = row.rating

director_dict = {}
for i, row in director.iterrows():
   director_dict[row.Director_Name] = row.rating


In [ ]:
def comparaison_string(str1, str2): #Renvoie la ressemblance en deux strings
    S = SequenceMatcher(lambda x: x in " \t",  str1, str2)
    if S.ratio() > 0.8 :
        return True
    else :
        return False

def comparaison_dict(string, dict): #Renvoie le rating associé à un acteur
    for name, rating in dict.items():
        if comparaison_string(string, name):
            return rating
    return 0

In [ ]:
for i, row in df.iterrows():
    sum_actor = 0
    sum_director = 0
    for actor in row.casting:
        sum_actor += comparaison_dict(actor, actor_dict)
    for director in row.realisateur:
        sum_director += comparaison_dict(director, director_dict)
    
    row.casting = sum_actor
    row.realisateur = sum_director

df.head()